# EDTF support - demonstration and validation

This notebook demonstrates and validates `undate` support for specific parts of the [Extended Date/Time Format (EDTF)](https://www.loc.gov/standards/datetime/) specification.

This notebook follows the same structure and uses the example from the Library of Congress specification, demonstrating parsing EDTF dates and formating dates in EDTF syntax, for the parts of the specification undate implements.

Undate only handles dates and date intervals; time is not supported.

*Notebook authored by Rebecca Sutton Koeser, October 2024.*

## Level 0

Full support for **Date** and **Time Interval**; **Date and Time** is not supported.

### Date

```
complete representation:            [year][“-”][month][“-”][day]
Example 1          ‘1985-04-12’ refers to the calendar date 1985 April 12th with day precision.
reduced precision for year and month:   [year][“-”][month]
Example 2          ‘1985-04’ refers to the calendar month 1985 April with month precision.
reduced precision for year:  [year]
Example 3          ‘1985’ refers to the calendar year 1985 with year precision.
```

#### Parse EDTF format

Demonstrate that these EDTF strings can be parsed into `Undate` objects.

In [44]:
from datetime import date

# Example 1: day
day = Undate.parse("1985-04-12", "EDTF")
assert day.precision == DatePrecision.DAY
assert day == date(1985, 4, 12)

# Example 2 : month
month = Undate.parse("1985-04", "EDTF")
assert month.year == "1985" and month.month == "04"
assert month.precision == DatePrecision.MONTH

# Example 3  : year
year = Undate.parse("1985", "EDTF")
assert year.year == "1985"
assert year.precision == DatePrecision.YEAR

#### Output in EDTF format

Demonstrate that initalizing `Undate` objects and serializing with EDTF formatter returns the expected value.

In [29]:
from undate.undate import Undate, DatePrecision
from undate.dateformat.edtf import EDTFDateFormat

# set default format to EDTF
Undate.DEFAULT_FORMAT = "EDTF"

# Example 1: day
day = Undate(1985, 4, 12)
# confirm EDTF formatter is being used
assert isinstance(day.formatter, EDTFDateFormat)
# casting to str is now equivalent to day.format("EDTF")
assert str(day) == "1985-04-12"
assert day.precision == DatePrecision.DAY

# Example 2 : month
month = Undate(1985, 4)
assert str(month) == "1985-04"
assert month.precision == DatePrecision.MONTH

# Example 3  : year
year = Undate(1985)
assert str(year) == "1985"
assert year.precision == DatePrecision.YEAR

### Date and Time - unsupported

`undate` does not includes support for time, onnly dates and date intervals.

### Time Interval

EDTF Level 0 adopts representations of a time interval where both the start and end are dates: start and end date only; that is, both start and duration, and duration and end, are excluded. Time of day is excluded.

```
    Example 1          ‘1964/2008’ is a time interval with calendar year precision, beginning sometime in 1964 and ending sometime in 2008.
    Example 2          ‘2004-06/2006-08’ is a time interval with calendar month precision, beginning sometime in June 2004 and ending sometime in August of 2006.
    Example 3          ‘2004-02-01/2005-02-08’ is a time interval with calendar day precision, beginning sometime on February 1, 2004 and ending sometime on February 8, 2005.
    Example 4          ‘2004-02-01/2005-02’ is a time interval beginning sometime on February 1, 2004 and ending sometime in February 2005. Since the start endpoint precision (day) is different than that of the end endpoint (month) the precision of the time interval at large is undefined.
    Example 5          ‘2004-02-01/2005’ is a time interval beginning sometime on February 1, 2004 and ending sometime in 2005. The start endpoint has calendar day precision and the end endpoint has calendar year precision. Similar to the previous example, the precision of the time interval at large is undefined.
    Example 6          ‘2005/2006-02’ is a time interval beginning sometime in 2005 and ending sometime in February 2006.
```

#### Parse EDTF format

In [57]:
from undate.undate import UndateInterval

# Example 1
year_range = Undate.parse("1964/2008", "EDTF")
assert isinstance(year_range, UndateInterval)
assert year_range.earliest == Undate(1964)
assert year_range.latest == Undate(2008)
# Example 2
month_range = Undate.parse("2004-06/2006-08", "EDTF")
assert isinstance(month_range, UndateInterval)
assert month_range.earliest == Undate(2004, 6)
assert month_range.latest == Undate(2006, 8)
# Example 3
day_range = Undate.parse("2004-02-01/2005-02-08", "EDTF")
assert isinstance(day_range, UndateInterval)
assert day_range.earliest == Undate(2004, 2, 1)
assert day_range.latest == Undate(2005, 2, 8)
# Example 4 
day_month_range = Undate.parse("2004-02-01/2005-02", "EDTF")
assert isinstance(day_range, UndateInterval)
assert day_month_range.earliest == Undate(2004, 2, 1)
assert day_month_range.latest == Undate(2005, 2)
assert day_month_range.earliest.precision == DatePrecision.DAY
assert day_month_range.latest.precision == DatePrecision.MONTH
# Example 5
day_year_range = Undate.parse("2004-02-01/2005", "EDTF")
assert isinstance(day_range, UndateInterval)
assert day_year_range.earliest == Undate(2004, 2, 1)
assert day_year_range.latest == Undate(2005)
assert day_year_range.earliest.precision == DatePrecision.DAY
assert day_year_range.latest.precision == DatePrecision.YEAR
# Example 6 
year_month_range = Undate.parse("2005/2006-02", "EDTF")
assert isinstance(year_month_range, UndateInterval)
assert year_month_range.earliest == Undate(2005)
assert year_month_range.latest == Undate(2006, 2)
assert year_month_range.earliest.precision == DatePrecision.YEAR
assert year_month_range.latest.precision == DatePrecision.MONTH


#### Output in EDTF format

In [58]:
from undate.undate import UndateInterval

# Example 1
assert UndateInterval(Undate(1964), Undate(2008)).format("EDTF") == "1964/2008"
# Example 2
assert UndateInterval(Undate(2004, 6), Undate(2006, 8)).format("EDTF") == "2004-06/2006-08"
# Example 3
assert UndateInterval(Undate(2004, 2, 1), Undate(2005, 2, 8)).format("EDTF") == "2004-02-01/2005-02-08"
# Example 4 
assert UndateInterval(Undate(2004, 2, 1), Undate(2005, 2)).format("EDTF") == "2004-02-01/2005-02"
# Example 5
assert UndateInterval(Undate(2004, 2, 1), Undate(2005)).format("EDTF") == "2004-02-01/2005"
# Example 6 
assert UndateInterval(Undate(2005), Undate(2006, 2)).format("EDTF") == "2005/2006-02"